# HMS-Commander (with Calibration Regions)
Author: William (Bill) Katzenmeyer, P.E., C.F.M. (C.H. Fenstermaker and Associates, LLC)

Source: https://github.com/billk-FM/HEC-Commander-

Changelog: 
2023-11-11 Raises error if hms_run_name not present in .run file
2024-01-30 User notification when restoring from backup file (to prevent overwrite loop)

In [ ]:
# User Defined Inputs

# Provide the HMS Project Directory and Project Name
hms_project_directory = r"C:\Your_HMS_Project_Directory"
hms_basin_file = "Your_HMS_Basin.basin"

# Define Calibration Runs CSV Filename (place in HMS Project Directory)
user_calibration_runs_csv_filename = "regions.csv"

hms_run_names = [
    "Your_Run_Name_1",
]
# If multiple run names are provided, each will generate a full set of user-defined runs

#Set a DSS output file suffix to differentiate run sets
hms_dss_suffix = "_Batch_Name"  # define your suffix here
print("DSS Suffix: " + hms_dss_suffix)

# Load the calibration shapefile
# To Create, export subbasins file from HEC-HMS, load in RASMapper, and merge subbasins.  
# Then, add "calregion" column and number calibration regions (1,2,3,etc.)
calibration_shapefile_path = 'C:\Calibration_Regions.shp' 

# Override Baseflow:None to Baseflow: Recession
hms_recession_baseflow = True

# Override Baseflow None to Baseflow Recession
Baseflow_Method_Set_to_Recession = "Yes"

In [ ]:
# ---- Additional Settings ----

# Define the path to the HEC-HMS executable
hms_executable_path = r"C:\Program Files\HEC\HEC-HMS\4.9"
print("HEC-HMS Executable Path: " + hms_executable_path)

# Jython Installation Path
jython_path = r"C:\jython2.7.3"
# This MUST match your Jython installation path or HMS will not run


# Define paths for HMScompute.py and HMScompute.bat which are used to run HMS through Jython
hms_compute_py_path = r"C:\jython2.7.3\HMScompute.py"
hms_compute_bat_path = r"C:\jython2.7.3\HMScompute.bat"

''' If jython heap size is too small, the HMS will freeze at 100% CPU and never finish'''
jython_initial_heap_size = "256m"       #initial heap size for java virtual machine
print("Jython Initial Heap Size: " + jython_initial_heap_size)
jython_maximum_heap_size = "4096m"      #maximum heap size for java virtual machine
print("Jython Maximum Heap Size: " + jython_maximum_heap_size)

# Set Canopy Method to Simple Zero to avoid warnings (Deprecated, not needed after HMS 4.9)
Canopy_Method_Set_To_Simple_Zero = "No"

In [ ]:
# Install/Import Required Python Packages
import sys
import subprocess


# List of packages you want to ensure are installed
packages = ["os", "shutil", "pandas", "geopandas", "subprocess", "re", "csv", "itertools"]


# Logic to Install Packages
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    print("Installed " + package + " successfully")

for package in packages:
    try:
        # If the import succeeds, the package is installed, so we can move on
        __import__(package)
    except ImportError:
        # If the import fails, the package is not installed and we try to install it
        install(package)


# Import All Required Libraries for Script
import shutil
import pandas as pd
import os
import time
import itertools
import re
import csv
import geopandas as gpd
from shapely.geometry import Point
from regex import D


# Check the Python version
python_version = sys.version_info

# Provide feedback if earlier Python version than 3.11 is used.
if python_version < (3, 11):
    raise SystemError("HMS-Commander's Calibration Regions relies on Geopandas and "
                      "may not work with python environments lower than 3.11")
else:
    print(f"Python version {python_version.major}.{python_version.minor}.{python_version.micro} is supported.")


In [ ]:
# Check for Jython 2.7.3 at C:\jython2.7.3

jython_jar_path = os.path.join(jython_path, "jython.jar")

def jython_exists(path):
    exists = os.path.exists(path)
    return exists

def print_installation_instructions():
    print("Jython is not found at the provided path.")
    print("Please install the necessary software:")
    print("1. Java SE Development Kit (check for the latest version): https://www.oracle.com/java/technologies/javase-jdk-downloads.html")
    print("   For HEC-HMS 4.9, use jdk-20.0.1  https://download.oracle.com/java/20/archive/jdk-20.0.1_windows-x64_bin.msi")
    print("2. Jython2.7.3: https://www.jython.org/download.html")
    print("   Direct Link: https://repo1.maven.org/maven2/org/python/jython-installer/2.7.3/jython-installer-2.7.3.jar")
    print("Be sure to install jython to the default location: C:\jython2.7.3 or change the jython_path variable in this script.")

if jython_exists(jython_jar_path):
    print(f"Jython Exists at {jython_jar_path}")
else:
    print_installation_instructions()
    raise FileNotFoundError(f"Jython not found at path: {jython_jar_path}")

# USER MUST INSTALL JYTHON 2.7.3 TO C:\jython2.7.3 OR CHANGE PATH IN THIS SCRIPT


In [ ]:
# Build HMScompute.bat and HMScompute.py for Jython 2.7.3

# Define the path and content for HMScompute.py
hms_compute_py_content = '''from hms.model import Project
from hms import Hms

import glob
import sys

hmspth = sys.argv[1]
runName = sys.argv[2]
# print 'running' + str(hmspth)

myProject = Project.open(hmspth)
myProject.computeRun(runName)
myProject.close()

Hms.shutdownEngine()
'''

# Define the path and content for HMScompute.bat
hms_compute_bat_content = f'''set "HMS={hms_executable_path}"
set "PATH=%HMS%\\bin\\gdal;%PATH%"
set "GDAL_DRIVER_PATH=%HMS%\\bin\\gdal\\gdalplugins"
set "GDAL_DATA=%HMS%\\bin\\gdal\\gdal-data"
set "PROJ_LIB=%HMS%\\bin\\gdal\\projlib"

set "CLASSPATH=%HMS%\\hms.jar;%HMS%\\lib\\*"
C:\\jython2.7.3\\bin\\jython.exe -J-Xms{jython_initial_heap_size} -J-Xmx{jython_maximum_heap_size} -Djava.library.path="%HMS%\\bin;%HMS%\\bin\\gdal" {hms_compute_py_path} %1 %2
'''

# Write the content for HMScompute.py
with open(hms_compute_py_path, 'w') as py_file:
    py_file.write(hms_compute_py_content)
print(f"Wrote content to {hms_compute_py_path}")


# Write the content for HMScompute.bat
with open(hms_compute_bat_path, 'w') as bat_file:
    bat_file.write(hms_compute_bat_content)
print(f"Wrote content to {hms_compute_bat_path}")

In [ ]:
#Logic to determine HMS Project Name and Required Paths

# Get HMS Project Name from hms_project_directory
def get_hms_project_name(hms_project_directory):
    """
    Given an HMS project directory, return the HMS project name.
    The project name is determined based on the .hms file in the directory.
    If more than one .hms file exists or none exist, an appropriate error message is returned.

    Args:
    - hms_project_directory (str): Path to the HMS project directory

    Returns:
    - str: HMS project name or error message
    """

    # List all files in the directory
    all_files = os.listdir(hms_project_directory)

    # Filter for files with .hms extension
    hms_files = [f for f in all_files if f.endswith('.hms')]

    # Check if there's exactly one .hms file
    if len(hms_files) == 1:
        # Extract the project name by stripping the .hms extension
        return hms_files[0].replace('.hms', '')
    elif len(hms_files) > 1:
        return "Error: More than one .hms file exists in the directory!"
    else:
        return "Error: No .hms file found in the directory!"

# Print Project Name or Error
hms_project_name = get_hms_project_name(hms_project_directory)
print("HMS Project Name: ", hms_project_name)


# Calculate and Print necessary paths and file names

# Calculated Paths and File Names (.run, .basin, backup paths)
user_calibration_runs_csv_fullpath = os.path.join(hms_project_directory, user_calibration_runs_csv_filename)
hms_project_run_file = f"{hms_project_name}.run"
hms_project_run_file_path = os.path.join(hms_project_directory, hms_project_run_file)
hms_grid_file = f"{hms_project_name}.grid"
hms_basin_file_path = os.path.join(hms_project_directory, hms_basin_file)
hms_project_run_file_backup_path = hms_project_run_file_path + ".bak"
hms_basin_file_backup_path = hms_basin_file_path + ".bak"

hms_grid_file_path = os.path.join(hms_project_directory, hms_grid_file)
hms_grid_file_backup_path = hms_grid_file_path + ".bak"

# Print statements
print("HMS User Calibration Runs CSV: ", user_calibration_runs_csv_fullpath)
print("HMS Project Run File:", hms_project_run_file)
print("HMS Project Run File Path:", hms_project_run_file_path)
print("HMS Basin File:", hms_basin_file)
print("HMS Basin File Path:", hms_basin_file_path)
print("HMS Project Run File Backup Path:", hms_project_run_file_backup_path)
print("HMS Basin File Backup Path:", hms_basin_file_backup_path)

In [ ]:
# Create Lock File and Logic for Working File Backup and Restore
# Define the path for the lock file based on one of the .bak file paths
lock_file_directory = os.path.dirname(hms_project_run_file_backup_path)
lock_file_path = os.path.join(lock_file_directory, "HMSCommander.lock")

# Step 1: Check if HMSCommander.lock exists
if os.path.exists(lock_file_path):
    user_confirmation = input("It appears the script did not run successfully and .bak files are still present "
                              "in the HMS folder. Please confirm that you want to restore the original "
                              ".basin, .run, and .grid files from backup. Type 'yes' or 'y' to confirm, "
                              "or any other key to cancel: ").lower()

    if user_confirmation in ['yes', 'y']:
        # Step 2: Restore files from backup
        for backup, original in [(hms_project_run_file_backup_path, hms_project_run_file_path),
                                 (hms_basin_file_backup_path, hms_basin_file_path),
                                 (hms_grid_file_backup_path, hms_grid_file_path)]:
            try:
                if os.path.exists(backup):
                    shutil.copy(backup, original)
                    # Delete the .bak file
                    os.remove(backup)
                    print(f"Restored and deleted file {backup}")
                else:
                    print(f"Backup file {backup} does not exist. Skipping restore.")
            except Exception as e:
                print(f"Error restoring file from {backup} to {original}: {e}")

        # Step 3: Delete the lock file
        os.remove(lock_file_path)
    else:
        print("Restoration process cancelled by user. Please inspect the files manually.")

else:
    print("Lock file does not exist. Proceeding to backup files.")

# Step 4: Backup files
for original, backup in [(hms_project_run_file_path, hms_project_run_file_backup_path),
                         (hms_basin_file_path, hms_basin_file_backup_path),
                         (hms_grid_file_path, hms_grid_file_backup_path)]:
    try:
        if os.path.exists(original):
            shutil.copy(original, backup)
            print(f"Backed up file from {original} to {backup}")
        else:
            print(f"Original file {original} does not exist. Skipping backup.")
    except Exception as e:
        print(f"Error backing up file from {original} to {backup}: {e}")

# Step 5: Create a new HMSCommander.lock file
with open(lock_file_path, 'w') as f:
    f.write("Lock file for HMS Commander operations.")

print("Lock file created.")

In [ ]:
# Read run parameters from CSV file
user_calibration_df = pd.read_csv(user_calibration_runs_csv_fullpath, dtype={'user_run_number_from_csv': int})

def load_user_calibration_csv_data(file_path):
    user_calibration_df = pd.read_csv(file_path, dtype={'user_run_number_from_csv': int})  # Directly reading csv using pandas to return DataFrame
    return user_calibration_df # Show DataFrame

load_user_calibration_csv_data(user_calibration_runs_csv_fullpath)


In [ ]:
# Define Basin File Prepreocessing Functions (Scale Factors, Recession Baseflow, etc.)

def update_baseflow_recession():
    """
    Updates the baseflow in hms_basin_file_data to Recession  Baseflow Method if set to "None".

    Parameters:
    - hms_basin_file_data: The initial basin file data
    - run: Dictionary containing the parameters 'recession_factor', 'initial_flow_area_ratio', and 'threshold_flow_to_peak_ratio'
    - hms_basin_file_path: Path to the basin file
    
    Returns:
    - Modified hms_basin_file_data
    """
    with open(hms_basin_file_path, 'r') as file:
        hms_basin_file_data = file.read()

    # Fill in Recession Baseflow if set as "None" and add required variables
    baseflow_none_text = "Baseflow: None"
    if baseflow_none_text in hms_basin_file_data:
        baseflow_text = "Baseflow: Recession\n Recession Factor: {}\n Initial Flow/Area Ratio: {}\n Threshold Flow to Peak Ratio: {}"
        baseflow_text = baseflow_text.format(run['recession_factor'], run['initial_flow_area_ratio'], run['threshold_flow_to_peak_ratio'])
        hms_basin_file_data = hms_basin_file_data.replace(baseflow_none_text, baseflow_text)
        print("Baseflow Set to None.  Inserting Recession Baseflow Parameters")  # Indicate that Baseflow data was filled

    with open(hms_basin_file_path, 'w') as file:
        file.write(hms_basin_file_data)
    return hms_basin_file_data

def modify_canopy_method(file_path):
    modified_lines = []
    with open(file_path, 'r') as file:
        lines = file.readlines()

    for line in lines:
        if 'Canopy: None' in line:
            modified_lines.append('    Canopy: Simple\n')
        elif 'Allow Simultaneous Precip Et: No' in line:
            modified_lines.append(line)
            modified_lines.append('    Initial Canopy Storage Percent: 1\n')
            modified_lines.append('    Canopy Storage Capacity: 0\n')
            modified_lines.append('    Crop Coefficient: 1.0\n')
            modified_lines.append('    End Canopy:\n')
        else:
            modified_lines.append(line)

    # Write back the modified content to the file
    with open(file_path, 'w') as file:
        file.writelines(modified_lines)

        
#Updates the grid file with the new DSS file path based on the impervious area scale.
def update_impervious_grid_definitions(grid_file_path, grid_def, dss_file_path, impervious_area_scale):
    
    with open(grid_file_path, 'r') as file:
        lines = file.readlines()

    grid_def_found = False
    for i, line in enumerate(lines):
        if f"Grid: {grid_def}" in line:
            grid_def_found = True
            print("Grid Definition Found!")
        if grid_def_found and "DSS File Name:" in line:
            old_dss_file_name = line.split(": ")[1].strip()
            new_dss_file_name = "data\\" + dss_file_path.split("\\")[-1]
            lines[i] = line.replace(old_dss_file_name, new_dss_file_name)
            print("DSS FileName Replaced!")
            break

    with open(grid_file_path, 'w') as file:
        file.writelines(lines)

#Finds the path of the Impervious DSS file based on the impervious area scale.
def find_impervious_dss_grids(directory, impervious_area_scale):
    
    scale_factor_to_filename = {
        1.2: "Impervious_1.2_SF.dss",
        1.4: "Impervious_1.4_SF.dss",
        1.6: "Impervious_1.6_SF.dss",
        1.8: "Impervious_1.8_SF.dss",
        2.0: "Impervious_2.0_SF.dss",
    }
    filename = scale_factor_to_filename.get(impervious_area_scale)
    if filename:
        full_path = os.path.join(directory, "data", filename)
        if os.path.isfile(full_path):
            return full_path
        else:
            print(f"Error: File '{filename}' cannot be found in the directory '{directory}/data'.")
            return None
    else:
        print(f"Error: No corresponding dss file found for the impervious area scale '{impervious_area_scale}'.")
        return None
        
# Reads the content of the HMS basin file
def read_basin_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    return content

# Optional: pauses the script execution until the user provides input.
def pause_script():
    
    user_input = input("Press Enter to continue or 'q' to quit: ")
    if user_input.lower() == 'q':
        exit()

# Optional: print the first 60 lines of a file
def print_first_60_lines(file_path):
    """Prints the first 60 lines of a specified file."""
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for i, line in enumerate(lines):
            if i < 60:
                print(line.strip())
            else:
                break

In [ ]:
# Define all basin update functions

# Loads the user calibration data from the specified file path into a pandas dataframe.
def load_subbasin_tc_data_to_pandas_dataframe(filepath):
    """Loads subbasin time of concentration and storage coefficient data into a pandas dataframe."""
    with open(filepath, 'r') as file:
        lines = file.readlines()

    subbasin_data = []
    current_subbasin = None
    toc = None
    sc = None

    for line in lines:
        if 'Subbasin:' in line:
            current_subbasin = line.split(': ')[1].strip()
        elif 'Time of Concentration:' in line:
            toc = float(line.split(': ')[1].strip())
        elif 'Storage Coefficient:' in line:
            sc = float(line.split(': ')[1].strip())
        elif 'End:' in line:
            if current_subbasin and toc and sc:
                subbasin_data.append([current_subbasin, toc, sc])
                current_subbasin, toc, sc = None, None, None

    subbasin_tc_r = pd.DataFrame(subbasin_data, columns=['subbasin_name', 'time_of_concentration', 'storage_coefficient'])
    return subbasin_tc_r

# Loads the user calibration data for the current run number into a pandas dataframe.
def fetch_calibration_parameters_for_current_run(user_calibration_df, current_run_number):
    """Fetches the calibration parameters for a specified run number."""
    current_calibration = user_calibration_df[user_calibration_df['user_run_number_from_csv'] == current_run_number]
    if not current_calibration.empty:
        return current_calibration.iloc[0]  
    else:
        print("No match found for current run number in user calibration data.")
        return None  # Return None if no matching calibration is found

In [ ]:
# Load Calibration Region Data 
# Function to load subbasin data
def load_all_subbasin_data(filepath):
    try:
        with open(filepath, 'r') as file:
            lines = file.readlines()
    except FileNotFoundError:
        print(f"The file {filepath} does not exist.")
        return pd.DataFrame()

    subbasin_data = []
    current_subbasin = {}
    collect_data = False

    for line in lines:
        line = line.strip()
        if not line:
            continue
        elif 'Subbasin:' in line:
            collect_data = True
            current_subbasin['Subbasin'] = line.split(': ')[1]
            #print(f"Processing subbasin: {current_subbasin['Subbasin']}")
        elif line == 'End:' and collect_data:
            collect_data = False
            subbasin_data.append(current_subbasin)
            current_subbasin = {}
        elif collect_data:
            parts = line.split(': ')
            if len(parts) == 2:
                key, value = parts
                current_subbasin[key] = value

    hms_subbasin_data_for_calibration_regions = pd.DataFrame(subbasin_data)
    return hms_subbasin_data_for_calibration_regions

# Load the subbasin data
print("Loading subbasin data")
hms_subbasin_data_for_calibration_regions_df = load_all_subbasin_data(hms_basin_file_path)

# Display the subbasin data
#display(hms_subbasin_data_for_calibration_regions_df)

# Write the subbasin data to a CSV file for reference
hms_subbasin_data_for_calibration_regions_df.to_csv(
    os.path.join(hms_project_directory, 'hms_subbasin_data_for_calibration_regions_df.csv'),
    index=False  # Avoid writing row names (index)
)

# Load the calibration shapefile
print("Loading calibration shapefile")
calibration_shapefile_gdf = gpd.read_file(calibration_shapefile_path)

# Converting the CRS of the shapefile to EPSG:4326 to match the subbasin coordinates
print("Converting CRS of calibration shapefile to EPSG:4326 (WSG84 to match lat/long coordinates)")
calibration_shapefile_gdf = calibration_shapefile_gdf.to_crs("EPSG:4326")

# Assigning the CRS of the subbasin GeoDataFrame to EPSG:4326
hms_subbasin_data_calregion_mapping_df = hms_subbasin_data_for_calibration_regions_df
hms_subbasin_data_calregion_mapping_df['geometry'] = hms_subbasin_data_calregion_mapping_df.apply(lambda row: Point(float(row['Longitude Degrees']), float(row['Latitude Degrees'])), axis=1)
hms_subbasin_data_calregion_mapping_gdf = gpd.GeoDataFrame(hms_subbasin_data_calregion_mapping_df, geometry='geometry')
hms_subbasin_data_calregion_mapping_gdf.crs = "EPSG:4326"

# Manually iterating over each subbasin point to find which calibration region it falls within
def find_cal_region(row):
    point = Point(float(row['Longitude Degrees']), float(row['Latitude Degrees']))
    for idx, region in calibration_shapefile_gdf.iterrows():
        if region['geometry'].contains(point):
            return region['CalRegion']
    return None

# Applying the function to each row in the DataFrame
hms_subbasin_data_calregion_mapping_df['CalRegion'] = hms_subbasin_data_calregion_mapping_df.apply(find_cal_region, axis=1)

# Reporting subbasins that do not have a calibration region
no_cal_region = hms_subbasin_data_calregion_mapping_df[hms_subbasin_data_calregion_mapping_df['CalRegion'].isnull()]

# Displaying the resulting DataFrame and subbasins with no calibration region
display(hms_subbasin_data_calregion_mapping_df[['Subbasin', 'Latitude Degrees', 'Longitude Degrees', 'CalRegion']])

# Assuming hms_subbasin_data_calregion_mapping_df is your DataFrame
hms_subbasin_data_calregion_mapping_df[['Subbasin', 'Latitude Degrees', 'Longitude Degrees', 'CalRegion']].to_csv(
    os.path.join(hms_project_directory, 'hms_subbasin_data_calregion_mapping_df.csv'),
    index=False  # Avoid writing row names (index)
)

In [ ]:
# Functions to Scale and Update Basin with Calibration Parameters by Calibration Region

'''" Example of In-Cell Context and Prompting 

Write a python function will take the existing hms_subbasin_data_for_calibration_regions dataframe and create a copy hms_subbasin_data_for_calibration_regions_scaled and edit it as follows:

hms_subbasin_data_calregion_mapping_df is the dataframe that contains all the subbasin data for all the calibration regions, before scaling

hms_subbasin_data_calregion_mapping_scaled_df is the dataframe that contains all the subbasin data for all the calibration regions, after scaling


Example Input CSV:  
user_run_number_from_csv,calregion,initial_deficit_scale,maximum_deficit_scale,percolation_rate_scale,impervious_area_scale,recession_factor,initial_flow_area_ratio,threshold_flow_to_peak_ratio,
1,1,0.99,0.91,0.1,0.85,0.11,1.01,0.15,0.9,1.1
1,2,0.98,0.92,0.05,0.84,0.12,1.02,0.14,0.8,1.2
1,3,0.97,0.93,0.01,0.83,0.13,1.03,0.13,0.7,1.3
2,1,0.96,0.94,0.02,0.82,0.14,1.04,0.12,0.6,1.4
2,2,0.95,0.95,0.03,0.81,0.15,1.05,0.11,0.5,1.5
2,3,0.94,0.96,0.04,0.8,0.26,1.06,0.1,0.4,1.6


Example hms_subbasin_data_calregion_mapping_df in CSV format:
Subbasin,Latitude Degrees,Longitude Degrees,CalRegion
S_TriggerBr_01, 30.68085406918698,-93.02073832096788,2
S_MuleyBr_01, 30.687537221284185,-93.00095193010225,2


Example hms_subbasin_data_calregion_mapping_df in CSV format:
Subbasin,Last Modified Date,Last Modified Time,Latitude Degrees,Longitude Degrees,Canvas X,Canvas Y,Area,Discretization,File,Projection,Cell Size,Canopy,Allow Simultaneous Precip Et,Plant Uptake Method,Initial Canopy Storage Percent,Canopy Storage Capacity,Crop Coefficient,Surface,LossRate,Initial Deficit Grid,Maximum Deficit Grid,Constant Rate Grid,Impervious Area Grid,Initial Deficit Scale,Maximum Deficit Scale,Percolation Rate Scale,Impervious Area Scale,Transform,Time of Concentration,Storage Coefficient,Baseflow,Recession Factor,Initial Flow/Area Ratio,Threshold Flow to Peak Ratio
S_TriggerBr_01,23-Sep-23,16:53:17,30.68085407,-93.02073832,2750381.379,797017.1065,1.7748,Structured,2018_Existing_Conditions.sqlite,5070,500,Simple,No,None,1,0,1,None,Gridded Deficit Constant,Initial_Moisture_Deficit,Max_Moisture_Deficit,Percolation_Rate,2019_Percent_Impervious,1,1,0.01,1,Modified Clark,10.77,2.69,Recession,0.1,1,0.1
S_MuleyBr_01,19-Jan-23,14:00:30,30.68753722,-93.00095193,2756636.354,799356.6226,2.8583,Structured,2018_Existing_Conditions.sqlite,5070,500,None,No,None,,,,None,Gridded Deficit Constant,Initial_Moisture_Deficit,Max_Moisture_Deficit,Percolation_Rate,2019_Percent_Impervious,1,1,0.01,1,Modified Clark,13.78,3.44,Recession,0.1,1,0.1


For each subbasin, the calibration region from hms_subbasin_data_calregion_mapping_df is used to determine the calibration region.  The corresponding input csv column is "calregion".

The columns that will be edited in hms_subbasin_data_calregion_mapping_df are:
Initial Deficit Scale       (directly copied from input csv)
Maximum Deficit Scale       (directly copied from input csv)
Percolation Rate Scale       (directly copied from input csv)
Impervious Area Scale (if <= 1.0)       (directly copied from input csv)
Recession Factor       (directly copied from input csv)
Initial Flow/Area Ratio       (directly copied from input csv)
Threshold Flow to Peak Ratio       (directly copied from input csv)
Time of Concentration       (use time_of_concentration_scale to scale the value from hms_subbasin_data_for_calibration_regions)
Storage Coefficient         (use storage_coefficient_scale to scale the value from hms_subbasin_data_for_calibration_regions)


Once the scaled dataframe has been edited, these values are written back to the .basin file.  Use this is a helpful function to emulate, its the one that populates hms_subbasin_data_for_calibration_regions.  The proposed function should run in reverse, and write only the needed lines from the dataframe back to the .basin file. 

def load_all_subbasin_data(filepath):
    try:
        with open(filepath, 'r') as file:
            lines = file.readlines()
    except FileNotFoundError:
        print(f"The file {filepath} does not exist.")
        return pd.DataFrame()

    subbasin_data = []
    current_subbasin = {}
    collect_data = False

    for line in lines:
        line = line.strip()
        if not line:
            continue
        elif 'Subbasin:' in line:
            collect_data = True
            current_subbasin['Subbasin'] = line.split(': ')[1]
            print(f"Processing subbasin: {current_subbasin['Subbasin']}")
        elif line == 'End:' and collect_data:
            collect_data = False
            subbasin_data.append(current_subbasin)
            current_subbasin = {}
        elif collect_data:
            parts = line.split(': ')
            if len(parts) == 2:
                key, value = parts
                current_subbasin[key] = value

    hms_subbasin_data_for_calibration_regions = pd.DataFrame(subbasin_data)
    return hms_subbasin_data_for_calibration_regions

Here is an example .basin file for example:

Basin: 2018_Existing_Conditions
Description: Existing Conditions based on USGS 2018 LiDAR
Last Modified Date: 1 September 2022
Last Modified Time: 19:53:15
Version: 4.9
Filepath Separator: \
Unit System: English
Missing Flow To Zero: No
Enable Flow Ratio: No
Compute Local Flow At Junctions: No
Unregulated Output Required: No

Enable Sediment Routing: No

Enable Quality Routing: No
End:

Subbasin: S_TriggerBr_01
Last Modified Date: 23 September 2023
Last Modified Time: 16:53:17
Latitude Degrees:  30.68085406918698
Longitude Degrees: -93.02073832096788
Canvas X: 2750381.3786021993
Canvas Y: 797017.1065448882
Area: 1.7748

Discretization: Structured
File: 2018_Existing_Conditions.sqlite
Projection: 5070
Cell Size: 500.0

Canopy: Simple
Allow Simultaneous Precip Et: No
Plant Uptake Method: None
Initial Canopy Storage Percent: 1
Canopy Storage Capacity: 0
Crop Coefficient: 1.0
End Canopy:

Surface: None

LossRate: Gridded Deficit Constant
Initial Deficit Grid: Initial_Moisture_Deficit
Maximum Deficit Grid: Max_Moisture_Deficit
Constant Rate Grid: Percolation_Rate
Impervious Area Grid: 2019_Percent_Impervious
Initial Deficit Scale: 1.0
Maximum Deficit Scale: 1.0
Percolation Rate Scale: 0.01
Impervious Area Scale: 1.0

Transform: Modified Clark
Time of Concentration: 10.77
Storage Coefficient: 2.69

Baseflow: Recession
Recession Factor: 0.1
Initial Flow/Area Ratio: 1.0
Threshold Flow to Peak Ratio: 0.1
End:

Subbasin: S_MuleyBr_01
Last Modified Date: 19 January 2023
Last Modified Time: 14:00:30
Latitude Degrees:  30.687537221284185
Longitude Degrees: -93.00095193010225
Canvas X: 2756636.3536137324
Canvas Y: 799356.6225776641
Area: 2.8583

Discretization: Structured
File: 2018_Existing_Conditions.sqlite
Projection: 5070
Cell Size: 500.0

Canopy: None
Allow Simultaneous Precip Et: No
Plant Uptake Method: None

Surface: None

LossRate: Gridded Deficit Constant
Initial Deficit Grid: Initial_Moisture_Deficit
Maximum Deficit Grid: Max_Moisture_Deficit
Constant Rate Grid: Percolation_Rate
Impervious Area Grid: 2019_Percent_Impervious
Initial Deficit Scale: 1.0
Maximum Deficit Scale: 1.0
Percolation Rate Scale: 0.01
Impervious Area Scale: 1.0

Transform: Modified Clark
Time of Concentration: 13.78
Storage Coefficient: 3.44

Baseflow: Recession
Recession Factor: 0.1
Initial Flow/Area Ratio: 1.0
Threshold Flow to Peak Ratio: 0.1
End:
'''

def scale_subbasin_values_by_calregion_d(mapping_df, calib_data_df, run_number):
    # Filter the calib_data_df by the run number
    calib_data_df = calib_data_df[calib_data_df['user_run_number_from_csv'] == run_number]
    
    # Continue with the rest of the logic as before
    mapping_df['CalRegion'] = mapping_df['CalRegion'].astype('int64')
    merged_df = pd.merge(mapping_df, calib_data_df, left_on='CalRegion', right_on='calregion', how='left')
    
    # ... rest of the code remains unchanged

    #display(merged_df)
    # Create a new DataFrame to store the updated values
    updated_values = {
        'Subbasin': [],
        'Initial Deficit Scale': [],
        'Maximum Deficit Scale': [],
        'Percolation Rate Scale': [],
        'Impervious Area Scale': [],
        'Recession Factor': [],
        'Initial Flow/Area Ratio': [],
        'Threshold Flow to Peak Ratio': [],
        'Time of Concentration': [],
        'Storage Coefficient': []
    }

    # Iterate over the merged DataFrame and update the values
    for _, row in merged_df.iterrows():
        subbasin = row['Subbasin']
        updated_values['Subbasin'].append(subbasin)
        updated_values['Initial Deficit Scale'].append(row['initial_deficit_scale'])
        updated_values['Maximum Deficit Scale'].append(row['maximum_deficit_scale'])
        updated_values['Percolation Rate Scale'].append(row['percolation_rate_scale'])
        updated_values['Impervious Area Scale'].append(row['impervious_area_scale'])
        updated_values['Recession Factor'].append(row['recession_factor'])
        updated_values['Initial Flow/Area Ratio'].append(row['initial_flow_area_ratio'])
        updated_values['Threshold Flow to Peak Ratio'].append(row['threshold_flow_to_peak_ratio'])
        # ... These values are scaled per the scale factor, then rounded to 2 decimal places
        updated_values['Time of Concentration'].append(round(float(row['Time of Concentration']) * row['time_of_concentration_scale'], 2))
        updated_values['Storage Coefficient'].append(round(float(row['Storage Coefficient']) * row['storage_coefficient_scale'], 2))
     
    # Create the DataFrame
    updated_df = pd.DataFrame(updated_values)

    return updated_df


# Function to write values back to .basin file

def write_scaled_values_to_basin_file(scaled_df, basin_filepath):
    try:
        with open(basin_filepath, 'r') as file:
            lines = file.readlines()
    except FileNotFoundError:
        print(f"The file {basin_filepath} does not exist.")
        return

    with open(basin_filepath, 'w') as outfile:
        current_subbasin = None
        for i, line in enumerate(lines):
            if 'Subbasin:' in line:
                current_subbasin = line.split(': ')[1].strip()
                subbasin_row = scaled_df[scaled_df['Subbasin'] == current_subbasin]
                if not subbasin_row.empty:
                    subbasin_data = subbasin_row.iloc[0]
            elif current_subbasin:
                parts = line.split(': ')
                if len(parts) == 2:
                    key, _ = parts
                    key = key.strip()
                    if key in scaled_df.columns:
                        new_value = subbasin_data.get(key)
                        if new_value is not None:
                            lines[i] = f"{key}: {new_value}\n"
        outfile.writelines(lines)


In [ ]:
# Main Logic of Script to Run HMS for each Event and Calibration Run

# Convert hms_run_names to a set to remove duplicates and then convert it back to a list
unique_hms_run_names = list(set(hms_run_names))
print(f"Unique HMS Run Names: {unique_hms_run_names}")

# Now iterate over the unique run names
for run_name in unique_hms_run_names:
    hms_run_name = run_name  # Set the current run name
    user_calibration_df['user_run_number_from_csv'] = user_calibration_df['user_run_number_from_csv'].astype(int)

    # For each calibration run, the script will:
    for _, run in user_calibration_df.iterrows():
        
        run = run.rename({k: k.lower() for k in run.index})
        current_run_number = int(run['user_run_number_from_csv'])

        print(f"Processing run number: {current_run_number}")  # Indicate which run is being processed

        hms_run_output_dss = f"{hms_run_name}_HMS_Run_{current_run_number}{hms_dss_suffix}.dss"
        hms_run_output_dss_path = os.path.join(hms_project_directory, f"{hms_run_output_dss}")

        # Backup the .grid file
        shutil.copy(hms_grid_file_path, hms_grid_file_backup_path)
        print("Backup of .grid file created.")  # Indicate that backup of .grid file has been made

        if not os.path.isfile(hms_run_output_dss_path):
            print("Output DSS file does not exist. Preparing HMS Run.")  

            # Update the .run file with the new DSS file name
            with open(hms_project_run_file_path, 'r') as file:
                run_data = file.readlines()

            run_found = False
            old_dss_file_name = None
            for i, line in enumerate(run_data):
                if f"Run: {hms_run_name}" in line:
                    run_found = True
                if "End:" in line:
                    run_found = False
                if run_found and line.strip().startswith("DSS File:"):
                    old_dss_file_name = line.split(":")[1].strip()
                    run_data[i] = line.replace(old_dss_file_name, hms_run_output_dss)
            
            if old_dss_file_name is None:
                raise Exception("hms_run_name not found in the HMS .run file. Check the run name in the User Input Section")
            else:
                print(f"Output DSS file name changed from {old_dss_file_name} to {hms_run_output_dss} in .run file.")

            # Write the updated .run data
            with open(hms_project_run_file_path, 'w') as file:
                file.writelines(run_data)


            # The condition is checked outside the function and the function is called if the condition is met
            if Canopy_Method_Set_To_Simple_Zero == "Yes":
                modify_canopy_method(hms_basin_file_path) # make sure hms_basin_file_path is defined


            # The main logic to check the condition before updating baseflow recession parameters
            if hms_recession_baseflow and Baseflow_Method_Set_to_Recession == "Yes":
                # Assuming you have already defined and set the variables hms_basin_file_data, run, and hms_basin_file_path
                update_baseflow_recession()
            else:
                print("Baseflow already set, or Baseflow_Method_Set_to_Recession is not set to 'Yes'. No changes made.")

            hms_subbasin_data_calregion_mapping_scaled_df = scale_subbasin_values_by_calregion_d(hms_subbasin_data_calregion_mapping_df, user_calibration_df, current_run_number)
            print("Scaled subbasin values by calibration region")
            display(hms_subbasin_data_calregion_mapping_scaled_df)


            write_scaled_values_to_basin_file(hms_subbasin_data_calregion_mapping_scaled_df, hms_basin_file_path)
            print("Done writing scaled values to .basin file")

            # THIS LOGIC NEEDS TO BE UPDATED SINCE WE HAVE 3 CALIBRATION REGION ENTERED IN THE CSV FILE
            # FOR NOW, DO NOT VARY IMPERVIOUS AREA SCALING BETWEEN >1.0 AND <1.0 - and if >1,0, use a single value for all calibration regions
            # Check if ANY Impervious Area Scale is greater than 1 and update the .grid file if necessary
            if run['impervious_area_scale'] > 1:
                dss_file_path = find_impervious_dss_grids(hms_project_directory, run['impervious_area_scale'])
                if dss_file_path:
                    update_impervious_grid_definitions(hms_grid_file_path, "2019_Percent_Impervious", dss_file_path, run['impervious_area_scale'])
                else:
                    # Restore files from backup and exit the script if the appropriate DSS file is not found
                    shutil.copy(hms_project_run_file_backup_path, hms_project_run_file_path)
                    shutil.copy(hms_basin_file_backup_path, hms_basin_file_path)
                    shutil.copy(hms_grid_file_backup_path, hms_grid_file_path)
                    print("Impervious DSS File Not Found. Restoring files and Exiting.")
                    exit()

            # Pause then execute HMS
            time.sleep(5)

            cmd = ['cmd', '/c', hms_compute_bat_path, os.path.join(hms_project_directory, hms_project_name + '.hms'), hms_run_name]
            print(f"Executing HMS with command: {' '.join(cmd)}")

            # Execute HMS
            subprocess.run(cmd)
            
            # HEC-HMS creates a log file with the hms_run_name with a .log extension
            # To avoid overwriting the log file, we rename it to include the run number and the suffix (matching the DSS file name with a .log extension)
            logfile_path = os.path.join(hms_project_directory, hms_run_output_dss.replace('.dss', '.log'))

            # Remove existing log file, if it exists
            if os.path.exists(logfile_path):
                os.remove(logfile_path)

            # Now rename the default HMS log file to match DSS file naming convention
            os.rename(os.path.join(hms_project_directory, hms_run_name + '.log'), logfile_path) if os.path.exists(os.path.join(hms_project_directory, hms_run_name + '.log')) else print(f"The file {os.path.join(hms_project_directory, hms_run_name + '.log')} does not exist.")
            print(f"Renamed log file to {logfile_path}")
            
                
            # Restore original HMS files
            shutil.copy(hms_basin_file_backup_path, hms_basin_file_path)
            print(f"Restored {hms_basin_file_backup_path} to {hms_basin_file_path}")
            shutil.copy(hms_grid_file_backup_path, hms_grid_file_path)
            print(f"Restored {hms_grid_file_backup_path} to {hms_grid_file_path}")
            print("")
        else:
            print(f"Run Output DSS file for Run {current_run_number} already exists.")
            print("")

    print ("All Runs Completed")

    # Delete the lock file for file backup and restoration
    if os.path.exists(lock_file_path):
        os.remove(lock_file_path)
        print("Lock file deleted.")
    else:
        print("Lock file does not exist.")

        
    print(f"All Runs Completed for {run_name}")
    
print ("All Events Processed")
